In [12]:
import json
from PIL import Image
import os
import pickle

In [13]:
# Loading data from the JSON file
with open("D:\IIT-D\Sem-4\JCD893-M.Tech Major Project Part-2\Anchor_Frame\Dataset\Metadata/mirgu_formatted_data.json", "r") as json_file:
    data = json.load(json_file)

In [14]:
# Access the loaded data from the JSON file

# List containing timestamp and converted binary data
data_array = data["data_array"]

# List containing frame types (0 for benign frames, 1 for attack frames)
frame_type = data["frame_type"]

# List containing unique converted binary data string for anchor frames
anchor = data["anchor"]

# Initialize an empty list to store image anchor numbers
image_anchor_number = []


In [15]:
def calculate_interframe_bits_new(frame, timestamp_difference, data_rate, i):
    """
    Calculating the number of interframe bits based on frame parameters and timestamp difference.

    Args:
        frame (str): Binary representation of the frame.
        timestamp_difference (float): Time difference between current and previous frames.
        data_rate (int): Data rate of the CAN bus (bits per second).
        i (int): Index of the current frame.

    Returns:
        str: Binary representation of interframe bits.

    """

    # Calculating the length of the frame in bits
    length_of_frame = len(frame)

    # Calculating the duration of the frame in seconds
    frame_duration = length_of_frame / data_rate

    # Calculating the interframe time (time gap between current and previous frames)
    interframe_time = timestamp_difference - frame_duration

    # Calculating the number of interframe bits based on the interframe time and data rate
    interframe_bits = int(data_rate * interframe_time)

    # Generating a string of '2's representing the interframe bits
    return '2' * interframe_bits


In [16]:
# def make_image_array(data_array, frame_type, anchor, data_rate):
#     """
#     Generate binary image arrays based on CAN data.

#     Args:
#         data_array (list): List of lists containing timestamp and converted binary data.
#         frame_type (list): List containing frame types (0 for benign frames, 1 for attack frames).
#         anchor (list): List containing unique converted binary data string for anchor frames.
#         data_rate (int): Data rate of the CAN bus (bits per second).

#     Returns:
#         tuple: A tuple containing three elements:
#             - binary_matrix (list): List of binary image arrays.
#             - y1 (list): List of labels indicating if the frame is an attack frame (1) or not (0).
#             - stats (list): List of dictionaries with counts of benign frames, attack frames, and the ratio of attack frames to total frames for each image.
#     """

#     # Initialising empty lists and variables

#     # List to store binary image arrays
#     binary_matrix = []

#     # List to store labels indicating majority frames
#     y1 = []

#     # List to store statistics
#     stats = []

#     # Initialising a 128x128 matrix with '0's
#     a = [['0' for _ in range(128)] for _ in range(128)]

#     # Initialising index for iterating over data_array
#     i = 0

#     # Initialising row for the matrix
#     x = 0

#     # Initialising column for the matrix
#     y = 0

#     # Initialising flag to distinguish between benign frames and attack frame
#     flag = 0

#     # Initialising counters for benign and attack frames
#     benign_count = 0
#     attack_count = 0

#     # Initialising counter for total frames in an image
#     count = 0

#     # Fraction depicts the fraction of attack frames to be present of number of frames in the image.
#     # Useful for IDS classifying it as attack or benign
#     fraction = 0.20

#     #image fill fraction to continue anchor logic
#     img_fraction=127/4

#     anchor_1 = []
#     anchor_2 = []
#     anchor_3 = []

#     # Iterating over each entry in data_array
#     while i < 500000:
#         # Extracting binary string representation of the frame
#         bin_str = data_array[i][1]

#         # Incrementing frame count
#         count += 1

#         # Setting the flag if the frame is a data frame
#         if frame_type[i] == 1:
#             flag += 1
#             attack_count += 1
#         else:
#             benign_count += 1
        
#         #row< 31
#         if bin_str in anchor and (x>0 and x < (int(img_fraction*1))):
#             while x < (int(img_fraction*1)):
#                 while y>=0:
#                     a[x][y]='4'
#                     if y==127:
#                         y=0
#                         break
#                     y+=1
#                 x+=1
#             anchor_1.append(len(binary_matrix)+1)

#             # Iterating over each bit in the binary string
#             for bit in bin_str:
#                 # Setting pixel in the matrix to the corresponding bit
#                 a[x][y] = bit

#                 # Moving to the next column
#                 if y == 127:
#                     x += 1
#                     y = 0
#                 else:
#                     y += 1

#             if i < len(data_array) - 1:
#                 # Calculate interframe bits and add them to the matrix
#                 timestamp_difference = data_array[i + 1][0] - data_array[i][0]
#                 interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
#                 for bit in interframe_bits:
#                     a[x][y] = bit
#                     if y == 127:
#                         x += 1
#                         y = 0
#                     else:
#                         y += 1

#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)
#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Resetting matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0

#                         i += 1
#                         break

#             # Adding row completion bits to start the next frame in a new row
#             while y < 128 and y > 0:
#                 a[x][y] = '3'
#                 if y == 127:
#                     x += 1
#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Reset matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0
#                         break
#                     y = 0
#                     break
#                 y += 1
#             i += 1

#         # 31<row<63
#         elif bin_str in anchor and (x>(int(img_fraction*1) and x < (int(img_fraction*2)))):
#             while x < (int(img_fraction*2)):
#                 while y>=0:
#                     a[x][y]='4'
#                     if y==127:
#                         y=0
#                         break
#                     y+=1
#                 x+=1
#             anchor_2.append(len(binary_matrix)+1)
#             # Iterating over each bit in the binary string
#             for bit in bin_str:
#                 # Setting pixel in the matrix to the corresponding bit
#                 a[x][y] = bit

#                 # Moving to the next column
#                 if y == 127:
#                     x += 1
#                     y = 0
#                 else:
#                     y += 1

#             if i < len(data_array) - 1:
#                 # Calculate interframe bits and add them to the matrix
#                 timestamp_difference = data_array[i + 1][0] - data_array[i][0]
#                 interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
#                 for bit in interframe_bits:
#                     a[x][y] = bit
#                     if y == 127:
#                         x += 1
#                         y = 0
#                     else:
#                         y += 1

#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Resetting matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0

#                         i += 1
#                         break

#             # Adding row completion bits to start the next frame in a new row
#             while y < 128 and y > 0:
#                 a[x][y] = '3'
#                 if y == 127:
#                     x += 1
#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Reset matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0
#                         break
#                     y = 0
#                     break
#                 y += 1
#             i += 1

#         # 63<row<95
#         elif bin_str in anchor and (x>(int(img_fraction*2) and x < (int(img_fraction*3)))):
#             while x < (int(img_fraction*3)):
#                 while y>=0:
#                     a[x][y]='4'
#                     if y==127:
#                         y=0
#                         break
#                     y+=1
#                 x+=1
#             anchor_3.append(len(binary_matrix)+1)
#             # Iterating over each bit in the binary string
#             for bit in bin_str:
#                 # Setting pixel in the matrix to the corresponding bit
#                 a[x][y] = bit

#                 # Moving to the next column
#                 if y == 127:
#                     x += 1
#                     y = 0
#                 else:
#                     y += 1

#             if i < len(data_array) - 1:
#                 # Calculate interframe bits and add them to the matrix
#                 timestamp_difference = data_array[i + 1][0] - data_array[i][0]
#                 interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
#                 for bit in interframe_bits:
#                     a[x][y] = bit
#                     if y == 127:
#                         x += 1
#                         y = 0
#                     else:
#                         y += 1

#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Resetting matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0

#                         i += 1
#                         break

#             # Adding row completion bits to start the next frame in a new row
#             while y < 128 and y > 0:
#                 a[x][y] = '3'
#                 if y == 127:
#                     x += 1
#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Reset matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0
#                         break
#                     y = 0
#                     break
#                 y += 1
#             i += 1


#         # Frame is not an anchor frame
#         else:
#             # Iterating over each bit in the binary string
#             for bit in bin_str:
#                 # Setting pixel in the matrix to the corresponding bit
#                 a[x][y] = bit

#                 # Moving to the next column
#                 if y == 127:
#                     x += 1
#                     y = 0
#                 else:
#                     y += 1

#                 # Checking if end of the matrix is reached
#                 if x == 128:
#                     # Append matrix to binary_matrix
#                     binary_matrix.append(a)

#                     # Append label indicating majority frame to y1
#                     y1.append(1 if flag >= (1) else 0)

#                     # Append statistics for the image
#                     stats.append({
#                         'benign_count': benign_count,
#                         'attack_count': attack_count,
#                         'attack_ratio': attack_count / count
#                     })

#                     # Reset flag, counts, and count
#                     flag = 0
#                     benign_count = 0
#                     attack_count = 0
#                     count = 0

#                     # Resetting matrix and row, column indexes
#                     a = [['0' for _ in range(128)] for _ in range(128)]
#                     x = 0
#                     y = 0

#                     i += 1
#                     break

#             # Check if there are more frames
#             if i < len(data_array) - 1:
#                 # Calculate interframe bits and add them to the matrix
#                 timestamp_difference = data_array[i + 1][0] - data_array[i][0]
#                 interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
#                 for bit in interframe_bits:
#                     a[x][y] = bit
#                     if y == 127:
#                         x += 1
#                         y = 0
#                     else:
#                         y += 1

#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Resetting matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0

#                         i += 1
#                         break

#             # Adding row completion bits to start the next frame in a new row
#             while y < 128 and y > 0:
#                 a[x][y] = '3'
#                 if y == 127:
#                     x += 1
#                     if x == 128:
#                         # Appending matrix to binary_matrix as an image has been generated
#                         binary_matrix.append(a)

#                         # Appending label indicating frame type to y1
#                         y1.append(1 if flag >= (1) else 0)

#                         # Appending statistics for the image
#                         stats.append({
#                             'benign_count': benign_count,
#                             'attack_count': attack_count,
#                             'attack_ratio': attack_count / count
#                         })

#                         # Resetting flag, counts, and count
#                         flag = 0
#                         benign_count = 0
#                         attack_count = 0
#                         count = 0

#                         # Reset matrix and row, column indexes
#                         a = [['0' for _ in range(128)] for _ in range(128)]
#                         x = 0
#                         y = 0
#                         break
#                     y = 0
#                     break
#                 y += 1

#         i += 1

#     return binary_matrix, y1, stats, anchor_1,anchor_2,anchor_3


In [17]:
def make_image_array(data_array, frame_type, anchor, data_rate):
    """
    Generate binary image arrays based on CAN data.

    Args:
        data_array (list): List of lists containing timestamp and converted binary data.
        frame_type (list): List containing frame types (0 for benign frames, 1 for attack frames).
        anchor (list): List containing unique converted binary data string for anchor frames.
        data_rate (int): Data rate of the CAN bus (bits per second).

    Returns:
        tuple: A tuple containing three elements:
            - binary_matrix (list): List of binary image arrays.
            - y1 (list): List of labels indicating if the frame is an attack frame (1) or not (0).
            - stats (list): List of dictionaries with counts of benign frames, attack frames, and the ratio of attack frames to total frames for each image.
    """

    # Initialising empty lists and variables

    # List to store binary image arrays
    binary_matrix = []

    # List to store labels indicating majority frames
    y1 = []

    # List to store statistics
    stats = []

    # Initialising a 128x128 matrix with '0's
    a = [['0' for _ in range(128)] for _ in range(128)]

    # Initialising index for iterating over data_array
    i = 0

    # Initialising row for the matrix
    x = 0

    # Initialising column for the matrix
    y = 0

    # Initialising flag to distinguish between benign frames and attack frame
    flag = 0

    # Initialising counters for benign and attack frames
    benign_count = 0
    attack_count = 0

    # Initialising counter for total frames in an image
    count = 0

    # Fraction depicts the fraction of attack frames to be present of number of frames in the image.
    # Useful for IDS classifying it as attack or benign
    fraction = 0.20

    #image fill fraction to continue anchor logic
    img_fraction=127/4

    anchor_1 = []
    anchor_2 = []
    anchor_3 = []
    anchor_4 = []

    # Iterating over each entry in data_array
    while i < len(data_array):
        # Extracting binary string representation of the frame
        bin_str = data_array[i][1]

        # Incrementing frame count
        count += 1

        # Setting the flag if the frame is a data frame
        if frame_type[i] == 1:
            flag += 1
            attack_count += 1
        else:
            benign_count += 1
        
        #row< 31
        if bin_str in anchor and (x>0 and x < (int(img_fraction*1))):
            while x < (int(img_fraction*1)):
                while y>=0:
                    a[x][y]='4'
                    if y==127:
                        y=0
                        break
                    y+=1
                x+=1
            anchor_1.append(len(binary_matrix)+1)

            # Iterating over each bit in the binary string
            for bit in bin_str:
                # Setting pixel in the matrix to the corresponding bit
                a[x][y] = bit

                # Moving to the next column
                if y == 127:
                    x += 1
                    y = 0
                else:
                    y += 1

            if i < len(data_array) - 1:
                # Calculate interframe bits and add them to the matrix
                timestamp_difference = data_array[i + 1][0] - data_array[i][0]
                interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
                for bit in interframe_bits:
                    a[x][y] = bit
                    if y == 127:
                        x += 1
                        y = 0
                    else:
                        y += 1

                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)
                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Resetting matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0

                        i += 1
                        break

            # Adding row completion bits to start the next frame in a new row
            while y < 128 and y > 0:
                a[x][y] = '3'
                if y == 127:
                    x += 1
                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Reset matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0
                        break
                    y = 0
                    break
                y += 1
            i += 1

        # 31<row<63
        elif bin_str in anchor and (x>(int(img_fraction*1) and x < (int(img_fraction*2)))):
            while x < (int(img_fraction*2)):
                while y>=0:
                    a[x][y]='4'
                    if y==127:
                        y=0
                        break
                    y+=1
                x+=1
            anchor_2.append(len(binary_matrix)+1)
            # Iterating over each bit in the binary string
            for bit in bin_str:
                # Setting pixel in the matrix to the corresponding bit
                a[x][y] = bit

                # Moving to the next column
                if y == 127:
                    x += 1
                    y = 0
                else:
                    y += 1

            if i < len(data_array) - 1:
                # Calculate interframe bits and add them to the matrix
                timestamp_difference = data_array[i + 1][0] - data_array[i][0]
                interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
                for bit in interframe_bits:
                    a[x][y] = bit
                    if y == 127:
                        x += 1
                        y = 0
                    else:
                        y += 1

                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Resetting matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0

                        i += 1
                        break

            # Adding row completion bits to start the next frame in a new row
            while y < 128 and y > 0:
                a[x][y] = '3'
                if y == 127:
                    x += 1
                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Reset matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0
                        break
                    y = 0
                    break
                y += 1
            i += 1

        # 63<row<95
        elif bin_str in anchor and (x>(int(img_fraction*2) and x < (int(img_fraction*3)))):
            while x < (int(img_fraction*3)):
                while y>=0:
                    a[x][y]='4'
                    if y==127:
                        y=0
                        break
                    y+=1
                x+=1
            anchor_3.append(len(binary_matrix)+1)
            # Iterating over each bit in the binary string
            for bit in bin_str:
                # Setting pixel in the matrix to the corresponding bit
                a[x][y] = bit

                # Moving to the next column
                if y == 127:
                    x += 1
                    y = 0
                else:
                    y += 1

            if i < len(data_array) - 1:
                # Calculate interframe bits and add them to the matrix
                timestamp_difference = data_array[i + 1][0] - data_array[i][0]
                interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
                for bit in interframe_bits:
                    a[x][y] = bit
                    if y == 127:
                        x += 1
                        y = 0
                    else:
                        y += 1

                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Resetting matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0

                        i += 1
                        break

            # Adding row completion bits to start the next frame in a new row
            while y < 128 and y > 0:
                a[x][y] = '3'
                if y == 127:
                    x += 1
                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Reset matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0
                        break
                    y = 0
                    break
                y += 1
            i += 1

        # 95<row<127
        elif bin_str in anchor and (x>(int(img_fraction*3) and x < (int(img_fraction*4)))):
            while x < (int(img_fraction*4)):
                while y>=0:
                    a[x][y]='4'
                    if y==127:
                        y=0
                        break
                    y+=1
                x+=1
            anchor_4.append(len(binary_matrix)+1)
            # Iterating over each bit in the binary string
            for bit in bin_str:
                # Setting pixel in the matrix to the corresponding bit
                a[x][y] = bit

                # Moving to the next column
                if y == 127:
                    # Appending matrix to binary_matrix as an image has been generated
                    binary_matrix.append(a)

                    # Appending label indicating frame type to y1
                    y1.append(1 if flag >= (1) else 0)

                    # Appending statistics for the image
                    stats.append({
                        'benign_count': benign_count,
                        'attack_count': attack_count,
                        'attack_ratio': attack_count / count
                    })

                    # Resetting flag, counts, and count
                    flag = 0
                    benign_count = 0
                    attack_count = 0
                    count = 0

                    # Reset matrix and row, column indexes
                    a = [['0' for _ in range(128)] for _ in range(128)]
                    x = 0
                    y = 0
                    break
                else:
                    y += 1

            if (i < len(data_array) - 1) and y!=0:
                # Calculate interframe bits and add them to the matrix
                timestamp_difference = data_array[i + 1][0] - data_array[i][0]
                interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
                for bit in interframe_bits:
                    a[x][y] = bit
                    if y == 127:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Resetting matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0
                        break
                    else:
                        y += 1
            i += 1


        # Frame is not an anchor frame
        else:
            # Iterating over each bit in the binary string
            for bit in bin_str:
                # Setting pixel in the matrix to the corresponding bit
                a[x][y] = bit

                # Moving to the next column
                if y == 127:
                    x += 1
                    y = 0
                else:
                    y += 1

                # Checking if end of the matrix is reached
                if x == 128:
                    # Append matrix to binary_matrix
                    binary_matrix.append(a)

                    # Append label indicating majority frame to y1
                    y1.append(1 if flag >= (1) else 0)

                    # Append statistics for the image
                    stats.append({
                        'benign_count': benign_count,
                        'attack_count': attack_count,
                        'attack_ratio': attack_count / count
                    })

                    # Reset flag, counts, and count
                    flag = 0
                    benign_count = 0
                    attack_count = 0
                    count = 0

                    # Resetting matrix and row, column indexes
                    a = [['0' for _ in range(128)] for _ in range(128)]
                    x = 0
                    y = 0

                    i += 1
                    break

            # Check if there are more frames
            if (i < len(data_array) - 1) and (x or y):
                # Calculate interframe bits and add them to the matrix
                timestamp_difference = data_array[i + 1][0] - data_array[i][0]
                interframe_bits = calculate_interframe_bits_new(data_array[i], timestamp_difference, data_rate, i)
                for bit in interframe_bits:
                    a[x][y] = bit
                    if y == 127:
                        x += 1
                        y = 0
                    else:
                        y += 1

                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Resetting matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0

                        i += 1
                        break

            # Adding row completion bits to start the next frame in a new row
            while (y < 128 and y > 0) and (x or y):
                a[x][y] = '3'
                if y == 127:
                    x += 1
                    if x == 128:
                        # Appending matrix to binary_matrix as an image has been generated
                        binary_matrix.append(a)

                        # Appending label indicating frame type to y1
                        y1.append(1 if flag >= (1) else 0)

                        # Appending statistics for the image
                        stats.append({
                            'benign_count': benign_count,
                            'attack_count': attack_count,
                            'attack_ratio': attack_count / count
                        })

                        # Resetting flag, counts, and count
                        flag = 0
                        benign_count = 0
                        attack_count = 0
                        count = 0

                        # Reset matrix and row, column indexes
                        a = [['0' for _ in range(128)] for _ in range(128)]
                        x = 0
                        y = 0
                        break
                    y = 0
                    break
                y += 1

        i += 1

    return binary_matrix, y1, stats, anchor_1,anchor_2,anchor_3,anchor_4


In [18]:


def image_generation(binary_matrix, y1):    
    # Define a color mapping dictionary to map binary values to colors
    color_mapping = {
        # '4': (255,255,0),
        '4': (255,255,0),
        '3': (255, 0, 0),  # Red for row completion bits
        '2': (0, 255, 0),  # Green for interframe bits
        '1': (255, 255, 255),  # White for data bits
        '0': (0, 0, 0)  # Black for empty bits
    }

    # Specifying the base output folder
    base_output_folder = 'D:/IIT-D/Sem-4/JCD893-M.Tech Major Project Part-2/Anchor_Frame/Generated_Images/mirgu_dataset'

    # Creating the attack and benign subfolders
    attack_folder = os.path.join(base_output_folder, 'attack')
    benign_folder = os.path.join(base_output_folder, 'benign')
    os.makedirs(attack_folder, exist_ok=True)
    os.makedirs(benign_folder, exist_ok=True)

    # Initializing a counter for the image filenames
    count = 1

    # Iterating through each 2D list in the binary_matrix
    for idx, two_d_list in enumerate(binary_matrix):

        # Create a blank image with the size of 128x128 pixels
        image_size = (128, 128)
        img = Image.new('RGB', image_size)

        # Iterate through each row in the 2D list
        for i, row in enumerate(two_d_list):

            # Iterate through each element in the row
            for j, element in enumerate(row):

                # Get the color corresponding to the binary value from the color_mapping dictionary
                # Default to black if the element is not found
                color = color_mapping.get(element, (0, 0, 0))

                # Set the pixel color in the image at the specified (j, i) position
                img.putpixel((j, i), color)

        # Generating a unique filename for the image
        filename = f'image_{count}.png'

        # Incrementing the counter for the next image filename
        count += 1

        # Determine the folder to save the image based on the y1 value
        if y1[idx] == 1:
            output_path = os.path.join(attack_folder, filename)
        else:
            output_path = os.path.join(benign_folder, filename)

        # Saving the resulting image in the appropriate folder
        img.save(output_path)



In [19]:
#Formation of Images
binary_matrix, y1, stats,anchor_1,anchor_2,anchor_3,anchor_4 = make_image_array(data_array, frame_type,anchor, data_rate=1024000) #512000
image_generation(binary_matrix,y1)


In [20]:
print(len(binary_matrix),len(anchor_1),len(anchor_2),len(anchor_3),len(anchor_4))

12970 4114 10895 0 0


In [21]:
def calculate_averages(stats):
    """
    Calculate the average total number of frames in an image and the average number of attack frames across all images.

    Args:
        stats (list): List of dictionaries with counts of benign frames, attack frames, and the ratio of attack frames to total frames for each image.

    Returns:
        tuple: A tuple containing two elements:
            - avg_total_frames (float): Average total number of frames in an image.
            - avg_attack_frames (float): Average number of attack frames in an image.
    """
    total_frames = [stat['benign_count'] + stat['attack_count'] for stat in stats]
    attack_frames = [stat['attack_count'] for stat in stats]

    avg_total_frames = sum(total_frames) / len(total_frames) if total_frames else 0
    avg_attack_frames = sum(attack_frames) / len(attack_frames) if attack_frames else 0

    return avg_total_frames, avg_attack_frames


avg_total_frames, avg_attack_frames = calculate_averages(stats)

print(f"Average total number of frames in an image: {avg_total_frames}")
print(f"Average number of attack frames in an image: {avg_attack_frames}")


Average total number of frames in an image: 24.398689282960678
Average number of attack frames in an image: 0.8107170393215112


In [22]:
import numpy as np
print(np.count_nonzero(y1))

6493
